In [2]:
#read from zip file path = 'local_data/shlomo/stock_data_bgeM3_bgeLarge_miniLM.zip'
import zipfile, json
from pathlib import Path
import re
import numpy as np
import pandas as pd



zip_path = 'local_data/shlomo/' + "stock_data_bgeM3_bgeLarge_miniLM.zip"
all_files = []
i=0
# model_name = "all-MiniLM-L6-v2"
model_name = "bge-large-en-v1.5"
pattern = re.compile(
    rf".*{model_name}/[A-Z]{{1,5}}_\d+_Q\d+/embedding\.npy$"
)
# pattern = re.compile(r".*all-MiniLM-L6-v2\/[A-Z]{1,5}_\d+_Q\d+\/embedding\.npy$")
# create dataframe with columns: fileNme, embedding
df = pd.DataFrame(columns=['file', 'embedding'])

with zipfile.ZipFile(zip_path, 'r') as z:
    for name in z.namelist():
        if pattern.match(name):
            # print(name)
            with z.open(name) as f:
                loaded = np.load(f)
                file_name = name.split('/')[-2]
                print(file_name)  # get the file name without path and extension
                df.loc[i] = [file_name, loaded]
                i+=1

A_2023_Q1
A_2023_Q2
A_2023_Q3
A_2024_Q1
A_2024_Q2
A_2024_Q3
A_2025_Q1
A_2025_Q2
AA_2022_Q3
AA_2023_Q1
AA_2023_Q2
AA_2023_Q3
AA_2024_Q1
AA_2024_Q2
AA_2024_Q3
AA_2025_Q1
AA_2025_Q2
AAON_2022_Q3
AAON_2023_Q1
AAON_2023_Q2
AAON_2023_Q3
AAON_2024_Q1
AAON_2024_Q2
AAON_2024_Q3
AAON_2025_Q1
AAON_2025_Q2
AAP_2022_Q4
AAP_2023_Q2
AAP_2023_Q3
AAP_2023_Q4
AAP_2024_Q2
AAP_2024_Q3
AAP_2024_Q4
AAP_2025_Q2
AAP_2025_Q3
AAPL_2022_Q4
AAPL_2023_Q2
AAPL_2023_Q3
AAPL_2023_Q4
AAPL_2024_Q1
AAPL_2024_Q2
AAPL_2024_Q4
AAPL_2025_Q1
AAPL_2025_Q2
ABBV_2022_Q3
ABBV_2023_Q1
ABBV_2023_Q2
ABBV_2023_Q3
ABBV_2024_Q1
ABBV_2024_Q2
ABBV_2024_Q3
ABBV_2025_Q1
ABBV_2025_Q2
ABNB_2022_Q3
ABNB_2023_Q1
ABNB_2023_Q2
ABNB_2023_Q3
ABNB_2024_Q1
ABNB_2024_Q2
ABNB_2024_Q3
ABNB_2025_Q1
ABNB_2025_Q2
ABT_2022_Q3
ABT_2023_Q1
ABT_2023_Q2
ABT_2023_Q3
ABT_2024_Q1
ABT_2024_Q2
ABT_2024_Q3
ABT_2025_Q1
ABT_2025_Q2
ACGL_2022_Q3
ACGL_2023_Q1
ACGL_2023_Q2
ACGL_2023_Q3
ACGL_2024_Q1
ACGL_2024_Q2
ACGL_2024_Q3
ACGL_2025_Q1
ACGL_2025_Q2
ACHC_2022_Q3
ACHC_20

In [3]:
df.head()

,file,embedding
0,A_2023_Q1,"[-0.0128278565, 0.0020887305, -0.0233907, 0.02..."
1,A_2023_Q2,"[-0.013832093, 0.0010737184, -0.02439624, 0.02..."
2,A_2023_Q3,"[-0.011602132, 0.0031424684, -0.021432886, 0.0..."
3,A_2024_Q1,"[-0.013549599, -0.0010417446, -0.025676511, 0...."
4,A_2024_Q2,"[-0.013174574, -0.00056200975, -0.02517565, 0...."


In [4]:
import re
import numpy as np
import pandas as pd
from pathlib import Path

def _parse_from_filename(name: str):
    print('****name = ' , name)
    """
    מצפה לשם קובץ בפורמט: TICKER_YYYY_Qn.json (למשל: A_2023_Q1.json)
    מחזיר dict עם ticker/year/quarter/join_key.
    """
    # m = re.match(r"(?P<ticker>[A-Za-z.\-]+)_(?P<year>\d{4})_Q(?P<q>[1-4])\.json$", name, flags=re.IGNORECASE)
    m = re.match(r"(?P<ticker>[A-Za-z.\-]+)_(?P<year>\d{4})_Q(?P<q>[1-4])", name, flags=re.IGNORECASE)


    if not m:
        return {"file": name, "ticker": np.nan, "year": np.nan, "quarter": np.nan, "join_key": np.nan}
    ticker = m.group("ticker").upper()
    year = int(m.group("year"))
    q = f"Q{m.group('q')}"
    return {"file": name, "ticker": ticker, "year": year, "quarter": q, "join_key": f"{ticker}|{year}{q}"}

def _ensure_join_key_on_embeddings(df_emb: pd.DataFrame) -> pd.DataFrame:
    """
    מוודא של־df_emb יש עמודת join_key בפורמט TICKER|YYYYQn.
    אם אין עמודות ticker/year/quarter — נגזור משם הקובץ (file).
    """
    df = df_emb.copy()
    if "file" not in df.columns:
        raise ValueError("df_emb חייב להכיל עמודה 'file' עם שם הקובץ, למשל A_2023_Q1.json")
    parsed = df["file"].apply(_parse_from_filename).apply(pd.Series)
    # אל תשכתב ערכים אם קיימים כבר, רק תמלא חסרים
    for col in ["ticker", "year", "quarter", "join_key"]:
        if col not in df.columns:
            df[col] = parsed[col]
        else:
            df[col] = df[col].fillna(parsed[col])
    # join_key סופי
    df["join_key"] = df["join_key"].fillna(
        df["ticker"].astype(str).str.upper().str.strip() + "|" +
        df["year"].astype("Int64").astype(str) + df["quarter"].astype(str).str.upper()
    )
    return df

def _load_results_build_keys(results_path: str) -> pd.DataFrame:
    """
    טוען את A_data.csv ובונה join_key = TICKER|YYYYQn
    מצפה לעמודות: 'Ticker', 'Quarter Year', 'Quarter Month', 'Stock Result'
    """
    p = Path(results_path)
    if p.suffix.lower() in [".xlsx", ".xls"]:
        r = pd.read_excel(p)
    else:
        r = pd.read_csv(p)

    # נורמליזציה של Quarter Month לפורמט Q1/Q2/Q3/Q4
    r["Quarter Month"] = (
        r["Quarter Month"].astype(str).str.upper().str.replace(r"[^Q0-9]", "", regex=True)
    )
    # יצירת join_key
    r["join_key"] = (
        r["Ticker"].astype(str).str.upper().str.strip() + "|" +
        r["Quarter Year"].astype(int).astype(str) + r["Quarter Month"]
    )

    # נשמור גם עמודות שימושיות נוספות
    cols_keep = ["join_key", "Stock Result"]
    for extra in ["Change %", "Neutral Threshold %", "Date", "Report Period", "Published At", "Filing Date"]:
        if extra in r.columns:
            cols_keep.append(extra)

    return r[cols_keep].copy()

def attach_stock_result_from_A_data(df_emb: pd.DataFrame, results_path: str) -> pd.DataFrame:
    """
    מצרף עמודת 'Stock Result' (ועוד עמודות שימושיות אם קיימות) ל־df_emb,
    לפי join_key = TICKER|YYYYQn כפי שמופיע ב־A_data.csv.
    """
    left = _ensure_join_key_on_embeddings(df_emb)
    right = _load_results_build_keys(results_path)

    merged = left.merge(right, on="join_key", how="left")

    # אופציונלי: מיפוי לייבל בינארי לחיזוי
    def _to_label(x):
        if pd.isna(x): return np.nan
        s = str(x).strip().upper()
        if s == "UP": return 1
        if s == "DOWN": return 0
        # NEUTRAL נשאיר כ-NaN כדי שלא "נכפה" כיוון
        return np.nan

    if "Stock Result" in merged.columns:
        merged["movement_label"] = merged["Stock Result"].apply(_to_label)

    return merged


In [5]:
import pandas as pd

# אם כבר יש לך DataFrame של הדוחות (עם 'file' כמו A_2023_Q1.json):
# df_emb = pd.read_parquet("report_embeddings.parquet")  # או df_emb שקיים לך בזיכרון
df_emb = df
all_stocks_data  = 'local_data/stocks_data.csv'
df_with_result = attach_stock_result_from_A_data(
    df_emb,
    results_path=all_stocks_data   # הקובץ שהעלית
)

****name =  A_2023_Q1
****name =  A_2023_Q2
****name =  A_2023_Q3
****name =  A_2024_Q1
****name =  A_2024_Q2
****name =  A_2024_Q3
****name =  A_2025_Q1
****name =  A_2025_Q2
****name =  AA_2022_Q3
****name =  AA_2023_Q1
****name =  AA_2023_Q2
****name =  AA_2023_Q3
****name =  AA_2024_Q1
****name =  AA_2024_Q2
****name =  AA_2024_Q3
****name =  AA_2025_Q1
****name =  AA_2025_Q2
****name =  AAON_2022_Q3
****name =  AAON_2023_Q1
****name =  AAON_2023_Q2
****name =  AAON_2023_Q3
****name =  AAON_2024_Q1
****name =  AAON_2024_Q2
****name =  AAON_2024_Q3
****name =  AAON_2025_Q1
****name =  AAON_2025_Q2
****name =  AAP_2022_Q4
****name =  AAP_2023_Q2
****name =  AAP_2023_Q3
****name =  AAP_2023_Q4
****name =  AAP_2024_Q2
****name =  AAP_2024_Q3
****name =  AAP_2024_Q4
****name =  AAP_2025_Q2
****name =  AAP_2025_Q3
****name =  AAPL_2022_Q4
****name =  AAPL_2023_Q2
****name =  AAPL_2023_Q3
****name =  AAPL_2023_Q4
****name =  AAPL_2024_Q1
****name =  AAPL_2024_Q2
****name =  AAPL_2024_Q4
*

In [6]:
df_with_result

,file,embedding,ticker,year,quarter,join_key,Stock Result,Neutral Threshold %,Report Period,Published At,movement_label
0,A_2023_Q1,"[-0.0128278565, 0.0020887305, -0.0233907, 0.02...",A,2023,Q1,A|2023Q1,NEUTRAL,2.25,2023-01-31,2023-03-02 23:52:33+00:00,NaN
1,A_2023_Q2,"[-0.013832093, 0.0010737184, -0.02439624, 0.02...",A,2023,Q2,A|2023Q2,UP,2.07,2023-04-30,2023-05-26 21:28:42+00:00,1.0
2,A_2023_Q3,"[-0.011602132, 0.0031424684, -0.021432886, 0.0...",A,2023,Q3,A|2023Q3,NEUTRAL,2.52,2023-07-31,2023-08-30 22:47:03+00:00,NaN
3,A_2024_Q1,"[-0.013549599, -0.0010417446, -0.025676511, 0....",A,2024,Q1,A|2024Q1,NEUTRAL,2.26,2024-01-31,2024-03-04 23:49:08+00:00,NaN
4,A_2024_Q2,"[-0.013174574, -0.00056200975, -0.02517565, 0....",A,2024,Q2,A|2024Q2,UP,2.08,2024-04-30,2024-05-31 22:52:05+00:00,1.0
...,...,...,...,...,...,...,...,...,...,...,...
8159,ZTS_2024_Q1,"[-0.016140452, 0.0027648103, -0.021068087, 0.0...",ZTS,2024,Q1,ZTS|2024Q1,NEUTRAL,2.46,2024-03-31,2024-05-02 16:26:11+00:00,NaN
8160,ZTS_2024_Q2,"[-0.015037224, 0.0017947688, -0.025470935, 0.0...",ZTS,2024,Q2,ZTS|2024Q2,NEUTRAL,3.05,2024-06-30,2024-08-06 16:41:47+00:00,NaN
8161,ZTS_2024_Q3,"[-0.013756514, 0.0014510993, -0.023787387, 0.0...",ZTS,2024,Q3,ZTS|2024Q3,NEUTRAL,2.86,2024-09-30,2024-11-04 18:52:13+00:00,NaN
8162,ZTS_2025_Q1,"[-0.01816138, -0.001021175, -0.023106486, 0.02...",ZTS,2025,Q1,ZTS|2025Q1,DOWN,2.94,2025-03-31,2025-05-06 17:29:54+00:00,0.0


In [7]:
data = df_with_result[['embedding','movement_label']]
data.head()

,embedding,movement_label
0,"[-0.0128278565, 0.0020887305, -0.0233907, 0.02...",NaN
1,"[-0.013832093, 0.0010737184, -0.02439624, 0.02...",1.0
2,"[-0.011602132, 0.0031424684, -0.021432886, 0.0...",NaN
3,"[-0.013549599, -0.0010417446, -0.025676511, 0....",NaN
4,"[-0.013174574, -0.00056200975, -0.02517565, 0....",1.0


In [8]:
data = df_with_result[['embedding','movement_label']]
import ast
# data['embedding'] = data['embedding'].apply(lambda x: np.fromstring(x.strip("[]"), sep=' ').tolist())
embeddings_df = pd.DataFrame(data["embedding"].tolist())

# מחברים חזרה עם ה־label
final_df = pd.concat([embeddings_df, data["movement_label"]], axis=1)

In [9]:

# # שמירה
file_name_save = 'local_data/data_for_supervised/embeded_final_' + model_name + '.parquet'
final_df.to_parquet(file_name_save, index=False)
# final_df.to_csv("local_data/data_for_supervised/embede_final_mini_lm_l6.csv", index=False)